# Setup

ライブラリのインポート

In [ ]:
from flask import Flask, request
import random, json, requests
import pandas as pd

# line用ライブラリ
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, StickerSendMessage,
)

credentialsの取得

In [ ]:
df = pd.read_json("credentials.json")["line"]
channel_secret = df["channel_secret"]
access_token = df["channel_access_token"]

# おみくじ

In [ ]:
app = Flask(__name__)

line_bot_api = LineBotApi(access_token)
handler = WebhookHandler(channel_secret)

# LINE APIおよびWebhookの接続
line_bot_api = LineBotApi(access_token)
handler = WebhookHandler(channel_secret)

# Flaskのルート設定
@app.route("/")
def hello_world():
    return "hello world!"

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 「おみくじ」と打つとランダムで運勢が返ってくる
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == "おみくじ":
        omikuji = random.choice(["大吉", "中吉", "小吉", "吉", "凶", "大凶"])
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=omikuji)
        )
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=event.message.text)
        )


#------------------------
if __name__ == "__main__":
    app.run()